In [70]:
from common_import import *
import prettytable
result_dict = readpkl('result_dict.pkl')

In [142]:
def print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=True):
    import prettytable

    # PrettyTable 객체 생성 및 필드 설정
    pt = prettytable.PrettyTable()
    pt.field_names = ['Dataset', 'E1', 'E2', 'Checkpoint', 'Subset']


    # 데이터를 리스트에 수집
    data = []
    for key in checkpoint_list:
        if key not in result_dict.keys():
            print(f"Checkpoint {key}에 대한 결과가 없습니다.")
            continue
        for subset in result_dict[key].keys():
            #if 'WITH_RZ' in subset: continue
            if 'STEP' in subset: continue
            if 'TRAIN_UNIV' in subset: continue
            splited = subset.split('-')
            if 'CAM_NO_FACTOR' in splited[-1] or 'CANONICAL' in splited[-1]:
                dataset = splited[0]
            else:
                dataset = splited[0] + '-' + splited[-1]
            #print(dataset, type(dataset))
            e1 = float(result_dict[key][subset]['e1'])
            e2 = float(result_dict[key][subset]['e2'])
            data.append([dataset, e1, e2, key, subset])

    # 데이터를 Subset 1순위, checkpoint 2순위로 정렬
    data_sorted = sorted(data, key=lambda x: (x[0], len(x[3])))

    prev_dataset = ''
    for i, row in enumerate(data_sorted):
        if prev_dataset != '' and prev_dataset != row[0]:
            data_sorted[i-1].append(True)
        else:
            data_sorted[i-1].append(False)
        prev_dataset = row[0]

    # 정렬된 데이터를 테이블에 추가
    for row in data_sorted:
        pt.add_row([row[0], f'{row[1]:.2f}', f'{row[2]:.2f}', row[3], row[4]], divider=row[-1])

    # 테이블 출력
    print(pt)

    # 엑셀로 저장
    if save_excel:
        df = pd.DataFrame(pt.rows, columns=pt.field_names)
        df.to_excel(output_file, index=False)
        print(f"PrettyTable 데이터를 {output_file}로 저장했습니다!")

def seed_summary(result_dict, checkpoint_list, save_path_to_excel='', print_table=True):
    field_names_e1 = []
    field_names_e2 = []

    # 데이터를 리스트에 수집
    data = {}
    total_seed_list = []
    seed_list = []
    no_seed_list = []
    for key in checkpoint_list:
        if key not in result_dict.keys():
            print(f"Checkpoint {key}에 대한 결과가 없습니다.")
            no_seed = True
        else:
            no_seed = False
        if 'seed1' in key: seed = 1
        elif 'seed2' in key: seed = 2
        elif 'seed3' in key: seed = 3
        elif 'seed4' in key: seed = 4
        else: seed = 0
        total_seed_list.append(seed)
        field_names_e1.append(f'seed{seed}_e1')
        field_names_e2.append(f'seed{seed}_e2')
        if not no_seed:
            seed_list.append(seed)
            for subset in result_dict[key].keys():
                #if 'CANONICAL_PCL' in subset: continue
                if 'ROT' in subset: continue
                if 'TRAIN_UNIV' in subset: continue
                if subset not in data.keys():
                    data[subset] = {}
                data[subset][seed] = {}
                data[subset][seed]['e1'] = float(result_dict[key][subset]['e1'])
                data[subset][seed]['e2'] = float(result_dict[key][subset]['e2'])
        else:
            no_seed_list.append(seed)

    for subset in data.keys():
        for seed in no_seed_list:
            if seed not in data[subset].keys():
                data[subset][seed] = {'e1': np.NaN, 'e2': np.NaN}

    field_names_e1.append('Average_e1')
    field_names_e2.append('Average_e2')
    #print(seed_list)

    # PrettyTable 객체 생성 및 필드 설정
    pt = prettytable.PrettyTable()
    pt.field_names = ['Subset'] + field_names_e1 + field_names_e2 # ['Subset', 'seed0_e1', 'seed1_e1', 'seed2_e1', 'seed3_e1', 'seed4_e1', 'Average_e1', 'seed0_e2', 'seed1_e2', 'seed2_e2', 'seed3_e2', 'seed4_e2', 'Average_e2']
    #print(pt.field_names)
    #print(data.keys())

    rows = []
    for subset in data.keys():
        # print(subset)
        if   subset == '3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN'   : row = ['3DHP TRAIN']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6'       : row = ['3DHP TEST']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6_UNIV'  : row = ['3DHP TEST (univ)']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-ALL_TEST'        : row = ['FIT3D ALL']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-TR_S03'          : row = ['FIT3D S03']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-TS_S4710'        : row = ['FIT3D S4710']
        elif subset == 'H36M-GT-CAM_NO_FACTOR'                  : row = ['H36M S9,11']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-TR_S1_TS_S5678'   : row = ['H36M S5678']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-TR_S1'            : row = ['3DHP TRAIN EXCEPT S1']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-TR_S15_TS_S678'   : row = ['H36M S678']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-TEST_ALL'         : row = ['H36M ALL']
        ##
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN'  : row = ['3DHP TRAIN']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6'      : row = ['3DHP TEST']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6_UNIV' : row = ['3DHP TEST (univ)']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-ALL_TEST'       : row = ['FIT3D ALL']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ'                 : row = ['H36M S9,11']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S1_TS_S5678'  : row = ['H36M S5678']
        ##
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN'  : row = ['3DHP TRAIN']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_TS1_6'      : row = ['3DHP TEST']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_TS1_6_UNIV' : row = ['3DHP TEST (univ)']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-ALL_TEST'       : row = ['FIT3D ALL']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TR_S03'         : row = ['FIT3D S03']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TS_S4710'       : row = ['FIT3D S4710']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE'                 : row = ['H36M S9,11']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TR_S1_TS_S5678'  : row = ['H36M S5678']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TR_S1'           : row = ['3DHP TRAIN EXCEPT S1']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TR_S15_TS_S678'  : row = ['H36M S678']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL'        : row = ['H36M ALL']
        ##
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT-TEST_ALL_TRAIN'  : row = ['3DHP TRAIN']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT-TEST_TS1_6'      : row = ['3DHP TEST']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT-TEST_TS1_6_UNIV' : row = ['3DHP TEST (univ)']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT-ALL_TEST'       : row = ['FIT3D ALL']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT'                 : row = ['H36M S9,11']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_WITH_KVIRT-TR_S1_TS_S5678'  : row = ['H36M S5678']
        ##
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-TEST_ALL_TRAIN'  : row = ['3DHP TRAIN']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-TEST_TS1_6'      : row = ['3DHP TEST']
        elif subset == '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-TEST_TS1_6_UNIV' : row = ['3DHP TEST (univ)']
        elif subset == 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-ALL_TEST'       : row = ['FIT3D ALL']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ'                 : row = ['H36M S9,11']
        elif subset == 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-TR_S1_TS_S5678'  : row = ['H36M S5678']
        else: row = [subset]

        for seed in total_seed_list:
            row.append(data[subset][seed]['e1'])
        row.append(np.mean([data[subset][i]['e1'] for i in seed_list]))
        for seed in total_seed_list:
            row.append(data[subset][seed]['e2'])
        row.append(np.mean([data[subset][i]['e2'] for i in seed_list]))
        rows.append(row)

    rows = sorted(rows, key=lambda x: (x[0]))

    # 정렬된 데이터를 테이블에 추가
    for row in rows:
        temp = [row[0]]
        temp += [f'{row[i]:.2f}' for i in range(1, len(row))]
        pt.add_row(temp)
    if print_table: print(pt)
    if save_path_to_excel != '':
        # save to excel
        df = pd.DataFrame(pt.rows, columns=pt.field_names)
        df.to_excel(save_path_to_excel, index=False)
    return pt

def load_excel_and_print_pt(file_path, checkpoint):
    df = pd.read_excel(file_path)
    pt = prettytable.PrettyTable()
    pt.field_names = df.columns
    for row in df.values:
        pt.add_row(row)
    pt.add_column('checkpoint', [checkpoint for _ in range(len(df))])
    print(pt)
    return pt

def load_excels_merge_print_pt(checkpoints, file_paths, save_path_to_excel=''):
    dfs = [pd.read_excel(file_path) for file_path in file_paths]
    pt = prettytable.PrettyTable()
    pt.field_names = list(dfs[0].columns) + ['checkpoint']
    rows = []
    for df, checkpoint in zip(dfs, checkpoints):
        for row in df.values:
            rows.append(row.tolist()+[checkpoint])
    rows_sorted = sorted(rows, key=lambda x: (x[0], len(x[-1])))

    prev_subset = rows_sorted[0][0]
    divider = []
    for i, row in enumerate(rows_sorted):
        if prev_subset != row[0]:
            divider.append(True)
        else:
            divider.append(False)
        prev_subset = row[0]
    divider.append(True)
    for i, row in enumerate(rows_sorted):
        pt.add_row(row, divider=divider[i+1])
    print(pt)
    if save_path_to_excel != '':
        df = pd.DataFrame(pt.rows, columns=pt.field_names)
        df.to_excel(save_path_to_excel, index=False)
    return pt

# Compare Models

In [13]:
for checkpoint in result_dict.keys():
    for subset in result_dict[checkpoint].keys():
        #print(checkpoint, subset, result_dict[checkpoint][subset]['e1'], result_dict[checkpoint][subset]['e2'])
        if 'ALL_TEST' in subset and 'H36M' in subset:
            print(subset, checkpoint)
            #print(result_dict[checkpoint][subset]['e1'], result_dict[checkpoint][subset]['e2'])

### Trained on H36M S15678 

In [37]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering{seed}',
        f'MB_train_h36m_gt_cam_no_factor{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_centering{seed}',
        #f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original{seed}',
    ]
    output_file = f"output_h36m{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=True)
    print()

+----------------------+--------+--------+---------------------------------------------------------------------------------+------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                    Checkpoint                                   |                                 Subset                                 |
+----------------------+--------+--------+---------------------------------------------------------------------------------+------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 95.59  | 68.06  |                          MB_train_h36m_gt_cam_no_factor                         |                  3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                  |
| 3DHP-TEST_ALL_TRAIN  | 115.18 | 58.73  |                  MB_train_h36m_gt_cam_no_factor_input_centering                 |                  3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                  |
| 3DHP-TES

### H36M S1 TRAIN

In [39]:
delete_list = []
for key in result_dict['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678'].keys():
    if 'PCL' not in key: continue
    delete_list.append(key)
    print(key)

for key in delete_list:
    del result_dict['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678'][key]
    
savepkl(result_dict, 'result_dict.pkl')

3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6_UNIV
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S1_TS_S5678
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-ALL_TEST


In [41]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678{seed}',
        #f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_ts_s5678{seed}',
        #f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_tr_s1_ts_s5678{seed}',
    ]
    output_file = f"output_h36m_tr_s1_ts_s5678{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=True)
    print()

+----------------------+--------+-------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2  |                                           Checkpoint                                           |                                 Subset                                 |
+----------------------+--------+-------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 116.92 | 77.36 |                         MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678                          |                  3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                  |
| 3DHP-TEST_ALL_TRAIN  | 122.77 | 66.04 |                 MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678                  |              

In [25]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678{seed}',
        f'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s15_ts_s678{seed}',
    ]
    output_file = f"output_h36m_tr_s15_ts_s678{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=True)
    print()

Checkpoint MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678에 대한 결과가 없습니다.
+----------------------+--------+--------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                           Checkpoint                                           |                                 Subset                                 |
+----------------------+--------+--------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 104.27 | 74.88  |                         MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678                          |                  3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                  |
| 3DHP-TEST_ALL_TRAIN  | 82.56  | 63.64  | MB_train_h36m_gt_c

### FIT3D TR_S03

In [43]:
delete_list = []
for key in result_dict['MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03'].keys():
    if 'PCL' not in key: continue
    delete_list.append(key)
    print(key)

for key in delete_list:
    del result_dict['MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03'][key]
    
savepkl(result_dict, 'result_dict.pkl')

H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S1_TS_S5678
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S03
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN


In [44]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_fit3d_gt_cam_no_factor_tr_s03{seed}',
        #f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03{seed}',
        f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03{seed}',
    ]
    print(f"**** seed = {seed} ****")
    output_file = f"output_fit3d_tr_s03{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file)
    print()

**** seed =  ****
+----------------------+--------+--------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                        Checkpoint                                       |                                 Subset                                 |
+----------------------+--------+--------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 216.50 | 144.11 |                          MB_train_fit3d_gt_cam_no_factor_tr_s03                         |                  3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                  |
| 3DHP-TEST_ALL_TRAIN  | 178.32 | 128.28 | MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03 | 3DHP-GT-CAM_NO_FACTOR-INP

### FIT3D TS_S4710

In [46]:
delete_list = []
for key in result_dict['MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710'].keys():
    if 'PCL' not in key: continue
    delete_list.append(key)
    print(key)

for key in delete_list:
    del result_dict['MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710'][key]
    
savepkl(result_dict, 'result_dict.pkl')

H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S1_TS_S5678
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TS_S4710


In [47]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_fit3d_gt_cam_no_factor_ts_s4710{seed}',
        #f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710{seed}',
        f'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710{seed}',
    ]
    output_file = f"output_fit3d_ts_s4710{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file)
    print()

+----------------------+--------+--------+-------------------------------------------------------------------------------------------+---------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                         Checkpoint                                        |                                Subset                               |
+----------------------+--------+--------+-------------------------------------------------------------------------------------------+---------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 205.57 | 137.99 |                          MB_train_fit3d_gt_cam_no_factor_ts_s4710                         |                 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN                |
| 3DHP-TEST_ALL_TRAIN  | 165.81 | 114.84 | MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710 |  3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_R

### 3DHP S1

In [48]:
for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
    checkpoint_list = [
        f'MB_train_3dhp_gt_cam_no_factor_tr_s1{seed}',
        #f'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1{seed}',
        f'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1{seed}',
    ]
    print(f"**** seed = {seed} ****")
    output_file = f"output_3dhp_tr_s1{seed}.xlsx"
    print_and_save_result(result_dict, checkpoint_list, output_file)
    print()

**** seed =  ****
+----------------------+--------+--------+---------------------------------------------------------------------------------------+---------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                       Checkpoint                                      |                                Subset                               |
+----------------------+--------+--------+---------------------------------------------------------------------------------------+---------------------------------------------------------------------+
|   3DHP-TEST_TS1_6    | 69.46  | 47.56  |                          MB_train_3dhp_gt_cam_no_factor_tr_s1                         |                   3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6                  |
|   3DHP-TEST_TS1_6    | 66.44  | 47.83  | MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1 |    3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_R

# Seed Summary

## Trained on H36M S15678

In [130]:
checkpoint_list = [
    'MB_train_h36m_gt_cam_no_factor',
    'MB_train_h36m_gt_cam_no_factor_input_centering',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering',
]
excel_list = []
for checkpoint in checkpoint_list:
    #print(checkpoint)
    checkpoint_seed_list = []
    for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
        checkpoint_seed_list += [f'{checkpoint}{seed}']
    save_path_to_excel = f'output_{checkpoint}.xlsx'
    pt = seed_summary(result_dict, checkpoint_seed_list, save_path_to_excel, print_table=False)
    excel_list.append(save_path_to_excel)
    print()

pt = load_excels_merge_print_pt(checkpoint_list, excel_list, save_path_to_excel='output_h36m_s15678.xlsx')




+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+---------------------------------------------------------------------------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |                                    checkpoint                                   |
+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+---------------------------------------------------------------------------------+
|    3DHP TEST     |  97.87   |  86.18   |   93.5   |  92.77   |  95.96   |   93.26    |  61.34   |   56.7   |  58.65   |  58.62   |  60.82   |   59.23    |                          MB_train_h36m_gt_cam_no_factor                         |
|    3DHP TEST     |  107.49  |  108.54  

In [137]:
checkpoint_list = [
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_input_centering',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering',
]
excel_list = []
for checkpoint in checkpoint_list:
    #print(checkpoint)
    checkpoint_seed_list = []
    for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
        checkpoint_seed_list += [f'{checkpoint}{seed}']
    save_path_to_excel = f'output_{checkpoint}.xlsx'
    pt = seed_summary(result_dict, checkpoint_seed_list, save_path_to_excel, print_table=False)
    excel_list.append(save_path_to_excel)
    print()

pt = load_excels_merge_print_pt(checkpoint_list, excel_list, save_path_to_excel='output_h36m_s15678_ablation.xlsx')



Checkpoint MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_input_centering에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_input_centering_seed1에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_input_centering_seed2에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_input_centering_seed3에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_input_centering_seed4에 대한 결과가 없습니다.


+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+---------------------------------------------------------------------------------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Ave

## Trained on H36M S1

In [129]:
checkpoint_list = [
    'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678',
    'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678',
]
excel_list = []
for checkpoint in checkpoint_list:
    #print(checkpoint)
    checkpoint_seed_list = []
    for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
        checkpoint_seed_list += [f'{checkpoint}{seed}']
    save_path_to_excel = f'output_{checkpoint}.xlsx'
    pt = seed_summary(result_dict, checkpoint_seed_list, save_path_to_excel, print_table=False)
    excel_list.append(save_path_to_excel)
    print()

pt = load_excels_merge_print_pt(checkpoint_list, excel_list, save_path_to_excel='output_h36m_s1.xlsx')


Checkpoint MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678_seed4에 대한 결과가 없습니다.


+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+------------------------------------------------------------------------------------------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |                                           checkpoint                                           |
+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+------------------------------------------------------------------------------------------------+
|    3DHP TEST     |  111.85  |  111.78  |  113.9   |  116.84  |  113.04  |   113.48   |  72.21   |  71.51   |  73.64   |  75.02   |  72.11   

## Trained on H36M S15

In [123]:
checkpoint_list = [
    'MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678',
    'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s15_ts_s678',
]
excel_list = []
for checkpoint in checkpoint_list:
    #print(checkpoint)
    checkpoint_seed_list = []
    for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
        checkpoint_seed_list += [f'{checkpoint}{seed}']
    save_path_to_excel = f'output_{checkpoint}.xlsx'
    pt = seed_summary(result_dict, checkpoint_seed_list, save_path_to_excel, print_table=False)
    excel_list.append(save_path_to_excel)
    print()

pt = load_excels_merge_print_pt(checkpoint_list, excel_list, save_path_to_excel='output_h36m_s15.xlsx')

Checkpoint MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678_seed1에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678_seed2에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678_seed3에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678_seed4에 대한 결과가 없습니다.

Checkpoint MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678_seed1에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678_seed2에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678_seed3에 대한 결과가 없습니다.
Checkpoint MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678_seed4에 대한 결과가 없습니다.

Checkpoint MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s15_ts_s678_seed3에 대한 결과가 없습니다.

+------------------+----------+----------+----------+----------+----------+------------+----------+-----

## Trained on FIT3D S3

In [ ]:
checkpoint_list = [
    'MB_train_fit3d_gt_cam_no_factor_tr_s03',
    'MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03',
    'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03',
]
excel_list = []
for checkpoint in checkpoint_list:
    #print(checkpoint)
    checkpoint_seed_list = []
    for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
        checkpoint_seed_list += [f'{checkpoint}{seed}']
    save_path_to_excel = f'output_{checkpoint}.xlsx'
    pt = seed_summary(result_dict, checkpoint_seed_list, save_path_to_excel, print_table=False)
    excel_list.append(save_path_to_excel)
    print()

pt = load_excels_merge_print_pt(checkpoint_list, excel_list, save_path_to_excel='output_fit3d_s03.xlsx')


Checkpoint MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03에 대한 결과가 없습니다.
Checkpoint MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03_seed1에 대한 결과가 없습니다.
Checkpoint MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03_seed2에 대한 결과가 없습니다.
Checkpoint MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03_seed3에 대한 결과가 없습니다.
Checkpoint MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03_seed4에 대한 결과가 없습니다.


+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+-----------------------------------------------------------------------------------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |                                        checkpoint                                       |
+------------------+----------+----------+----------+----------+----------+----

## Trained on FIT3D S3,5,8,9,11

In [143]:
checkpoint_list = [
    'MB_train_fit3d_gt_cam_no_factor_ts_s4710',
    'MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710',
    'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710',
]
excel_list = []
for checkpoint in checkpoint_list:
    #print(checkpoint)
    checkpoint_seed_list = []
    for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
        checkpoint_seed_list += [f'{checkpoint}{seed}']
    save_path_to_excel = f'output_{checkpoint}.xlsx'
    pt = seed_summary(result_dict, checkpoint_seed_list, save_path_to_excel, print_table=False)
    excel_list.append(save_path_to_excel)
    print()

pt = load_excels_merge_print_pt(checkpoint_list, excel_list, save_path_to_excel='output_fit3d_s358911.xlsx')


Checkpoint MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710에 대한 결과가 없습니다.
Checkpoint MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710_seed1에 대한 결과가 없습니다.
Checkpoint MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710_seed2에 대한 결과가 없습니다.
Checkpoint MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710_seed3에 대한 결과가 없습니다.
Checkpoint MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710_seed4에 대한 결과가 없습니다.


+------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+-------------------------------------------------------------------------------------------+
|      Subset      | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |                                         checkpoint                                        |
+------------------+----------+----------+----------+----------+-

## Trained on 3DHP S1

In [124]:
checkpoint_list = [
    'MB_train_3dhp_gt_cam_no_factor_tr_s1',
    'MB_train_3dhp_gt_cam_no_factor_input_centering_tr_s1',
    'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1',
]
excel_list = []
for checkpoint in checkpoint_list:
    #print(checkpoint)
    checkpoint_seed_list = []
    for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
        checkpoint_seed_list += [f'{checkpoint}{seed}']
    save_path_to_excel = f'output_{checkpoint}.xlsx'
    pt = seed_summary(result_dict, checkpoint_seed_list, save_path_to_excel, print_table=False)
    excel_list.append(save_path_to_excel)
    print()

pt = load_excels_merge_print_pt(checkpoint_list, excel_list, save_path_to_excel='output_3dhp_s1.xlsx')




+----------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+---------------------------------------------------------------------------------------+
|        Subset        | seed0_e1 | seed1_e1 | seed2_e1 | seed3_e1 | seed4_e1 | Average_e1 | seed0_e2 | seed1_e2 | seed2_e2 | seed3_e2 | seed4_e2 | Average_e2 |                                       checkpoint                                      |
+----------------------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+------------+---------------------------------------------------------------------------------------+
|      3DHP TEST       |  69.46   |  71.25   |  72.63   |  71.14   |  69.12   |   70.72    |  47.56   |   50.4   |  50.61   |  51.84   |  50.25   |   50.13    |                          MB_train_3dhp_gt_cam_no_factor_tr_s1                         |
|

## Trained on 3DHP TRAIN

In [128]:
checkpoint_list = [
    'MB_train_3dhp_gt_cam_no_factor',
    'MB_train_3dhp_gt_cam_no_factor_input_centering',
    'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering',
]
excel_list = []
for checkpoint in checkpoint_list:
    print(checkpoint)
    checkpoint_seed_list = []
    for seed in ['', '_seed1', '_seed2', '_seed3', '_seed4']:
        checkpoint_seed_list += [f'{checkpoint}{seed}']
    save_path_to_excel = f'output_{checkpoint}.xlsx'
    pt = seed_summary(result_dict, checkpoint_seed_list, save_path_to_excel, print_table=False)
    excel_list.append(save_path_to_excel)
    print()

pt = load_excels_merge_print_pt(checkpoint_list, excel_list, save_path_to_excel='output_3dhp_train.xlsx')

MB_train_3dhp_gt_cam_no_factor
Checkpoint MB_train_3dhp_gt_cam_no_factor_seed1에 대한 결과가 없습니다.
Checkpoint MB_train_3dhp_gt_cam_no_factor_seed2에 대한 결과가 없습니다.
Checkpoint MB_train_3dhp_gt_cam_no_factor_seed3에 대한 결과가 없습니다.
Checkpoint MB_train_3dhp_gt_cam_no_factor_seed4에 대한 결과가 없습니다.

MB_train_3dhp_gt_cam_no_factor_input_centering
Checkpoint MB_train_3dhp_gt_cam_no_factor_input_centering_seed1에 대한 결과가 없습니다.
Checkpoint MB_train_3dhp_gt_cam_no_factor_input_centering_seed2에 대한 결과가 없습니다.
Checkpoint MB_train_3dhp_gt_cam_no_factor_input_centering_seed3에 대한 결과가 없습니다.
Checkpoint MB_train_3dhp_gt_cam_no_factor_input_centering_seed4에 대한 결과가 없습니다.

MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering
Checkpoint MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed1에 대한 결과가 없습니다.
Checkpoint MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed2에 대한 결과가 없습니다.
Checkpoint MB_train_3dhp_gt_cam_no_factor_input_from_canon

# Total

In [ ]:
summary_table = prettytable.PrettyTable()
summary_table.field_names = ['checkpoint', 'subset', 'MPJPE', 'P-MPJPE']
for checkpoint in result_dict.keys():
    for subset in result_dict[checkpoint].keys():
        e1 = result_dict[checkpoint][subset]['e1']
        e2 = result_dict[checkpoint][subset]['e2']
        row = [checkpoint, subset, f'{e1:.2f}', f'{e2:.2f}']
        summary_table.add_row(row)
print(summary_table)

+-------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+--------+---------+
|                                                                       checkpoint                                                                      |                                          subset                                          | MPJPE  | P-MPJPE |
+-------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+--------+---------+
|                        MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1                       |     H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXE

In [ ]:
checkpoint_list = [
    "MB_train_h36m_gt_cam_no_factor_with_canonical2",
    'MB_train_h36m_gt_cam_no_factor_with_canonical2_tr_s1_ts_s5678',
    'MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03',
    'MB_train_fit3d_gt_cam_no_factor_with_canonical2_ts_s4710',
]
output_file = f"output_with_canonical2.xlsx"
print_and_save_result(result_dict, checkpoint_list, output_file, save_excel=False)
print()

Checkpoint MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt에 대한 결과가 없습니다.
+---------------------+--------+--------+---------------------------------------------------------------+--------------------------------------+
|       Dataset       |   E1   |   E2   |                           Checkpoint                          |                Subset                |
+---------------------+--------+--------+---------------------------------------------------------------+--------------------------------------+
| 3DHP-TEST_ALL_TRAIN | 186.13 | 125.26 |     MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03    | 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 171.22 | 108.59 |    MB_train_fit3d_gt_cam_no_factor_with_canonical2_ts_s4710   | 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 115.10 | 57.43  |         MB_train_h36m_gt_cam_no_factor_with_canonical2        | 3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 122.79